# TP2-3

### Emparelhamentos

In [35]:
# Geração dos primos  q, p
bq     = 192                # tamanho em bits do primo "q"
bp     = 768                # tamanho minimo em bits do primo "p"

q = random_prime(2^bq-1,lbound=2^(bq-1))

t = q*3*2^(bp - bq)
while not is_prime(t-1):
    t = t << 1

p = t - 1

# Aneis e Corpos
Fp     = GF(p)                  # corpo primo com "p" elementos
R.<z>  = Fp[]                   # anel dos polinomios em "z" de coeficientes em Fp
f     = R(z^2 + z + 1)
Fp2.<z> = GF(p^2, modulus=f)   
# extensão de Fp de dimensão 2 cujo módulo é o polinómio "f"
# o polinómio "f"  é irredutivel, tem grau 2 e verifica  z^3 = 1 mod f
# se o ponto (x,y) verificar a equação y^2 = x^3 + 1, 
#      então o ponto (z*x,y) verifica a mesma equação

def trace(x):       # função linear que mapeia Fp2  em  Fp
    return x + x^p

# Curvas Elipticas supersingulares em Sagemath

# a curva supersingular sobre Fp2  definida pela equação  y^2 = x^3 + 1
E2 = EllipticCurve(Fp2, [0,1])

# ponto arbitrário  de ordem "q" em E2        
cofac = (p + 1)//q
G = cofac * E2.random_point()

# emparelhamento e oraculo DDHP

def phi(P):             # a isogenia que mapeia  (x,y)  ->  (z*x,y)
    (x,y) = P.xy()
    return E2(z*x,y)

def TateX(P,Q,l=1):      # o emparelhamento de Tate generalizado
    return P.tate_pairing(phi(Q), q, 2)^l

def ddhp(P,Q,R):        # o oraculo DDHP  que decide se (P,Q,R) é um triplo de DH
    return tateX(P,Q) == tateX(R,G)

### Configuração

#### Geração do segredo administrativo e chave pública administrativa
De acordo com o stardard *rfc5091*, a geração destes valores, nesta fase de *setup*, é um pouco diferente. No entanto, no nosso caso, acontece da seguinte forma:

- **Input**: parâmetro de segurança n (definido no emparelhamento)
- **Output**: chave pública e segredo administrativo s

**Passos**
- gerar s no intervalo 0..q-1 a partir de um PRG
- gerar chave pública administrativa a partir de g(s) (g: n -> n∗G)

In [36]:
def keyGen(n):

    # q já foi definido 
    # gerar s 
    s = 1 
    #nounce =
    #s = PRG(nounce)

    # gerar chave pública administrativa
    pub_key = s*G

    return (pub_key,s)

### Extração da chave privada

In [37]:
id = "sally@bad.com"

In [44]:
def h(inp):
    byte_string = inp.encode('utf-8')
    value = int.from_bytes(byte_string, byteorder='big')
    return value

$$key\;\gets\;(\,\vartheta \;d \gets \mathsf{ID}(\mathit{id})\;\centerdot\; s* d\,)$$
$$d \gets \mathsf{ID}(id)$$
$$\mathsf{ID} \colon \mathsf{Bytes} \to \mathbb{G}  \ \ \ \ \ \text{definido como} \quad \quad\mathsf{ID}(m) \equiv g(h(m))$$
$$g \colon \mathbb{Z} \to \mathbb{G}  \ \ \ \ \ \text{definido como} \quad \;g \colon n \mapsto n \ast G$$
$$h \colon \mathsf{Bytes} \to \mathbb{Z}$$

In [45]:
def keyExtract(id):
    
    d = h(id)*G
    return s*d

### Crifrar

In [22]:
def Xor(a,b):
    int_a = int(a)
    int_b = int(b)
    return int_a ^^ int_b  

In [23]:
def H(inp):
    return

Para cifrar, criamos os métodos auxiliares de *enc_in* e *enc_out* que se devem comportar da seguinte maneira:

$$\mathsf{in}(\mathit{id},x) \equiv \vartheta\,d\gets\mathsf{ID}(\mathit{id})\,\centerdot\,\vartheta\,v\gets\mathsf{Zr}\,\centerdot\,\vartheta\,a\gets H(v\oplus x)\,\centerdot\,\vartheta\,\mu\gets\mathbf{ex}(\beta,d,a)\,\centerdot\,\langle x,v,a,\mu\rangle$$

$$\mathsf{out}(x,v,a,\mu) \equiv \vartheta\,\alpha\gets g(a)\,\centerdot\,\vartheta\,v'\gets v\oplus f(\mu)\,\centerdot\,x'\gets x\oplus H(v)\,\centerdot\,\langle \alpha,v',x'\rangle$$


In [24]:
def enc_in(id,x):
    return

def enc_out(x,v,a,u):
    return

O próprio método para cifrar usa estes auxiliares da seguinte forma:
$$\mathsf{Encrypt}(\mathit{id}, x) \;\equiv\; \vartheta \,x,v,a,\mu \gets \mathsf{in}(\mathit{id},x)\,\centerdot\,\mathsf{out}(x,v,a,\mu)$$

In [25]:
def encrypt(id,x):
    
    inter = enc_in(id,x)
    out = enc_out(inter)

### Decifrar

In [47]:
key = keyExtract(id)

De forma semelhante, mas agora para decifrar, criamos os métodos auxiliares de *dec_in* e *dec_out* que se devem comportar da seguinte maneira:
$$\mathsf{in}(\mathit{key},\alpha, v', x') \equiv \vartheta\,\mu \gets \mathbf{ex}(\alpha,\mathit{key},1)\,\centerdot\,\vartheta\,v \gets v'\oplus f(\mu)\,\centerdot\,x\gets x'\oplus H(v)\,\centerdot\,\langle \alpha,v,x\rangle$$
$$\mathsf{out}(\alpha,v,x) \equiv \vartheta\,a\gets H(v\oplus x)\,\centerdot\,\mathsf{if}\;\;\alpha\neq g(a)\;\;\mathsf{then}\;\;\texttt{fails}\;\;\mathsf{else}\;\;x$$

In [ ]:
def dec_in():
    return

def dec_out():
    return

O método para decifrar funciona desta forma:
$$\mathsf{Decrypt}(\mathit{key}, c) \;\equiv\; \vartheta \,\alpha,v,x \gets \mathsf{in}(\mathit{key},c)\,\centerdot\,\mathsf{out}(\alpha,v,x)$$

In [ ]:
def decrypt():
    return